In [0]:
dbutils.fs.rm('dbfs:/FileStore/tables', True)

Out[2]: True

In [0]:
files_name = ['Categories','CustomerCustomerDemo','CustomerDemographics','Customers','Employees','EmployeeTerritories','Orders','Order_Details_','Products','Region','Shippers','Suppliers','Territories']
for file in files_name:
    file_location = "/FileStore/tables/"+file+".csv"
    file_type = 'csv'
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ','

    current_table = spark.read.format(file_type)\
    .option("inferSchema", infer_schema)\
    .option("header", first_row_is_header)\
    .option("sep", delimiter)\
    .option("multiLine", "true")\
    .load(file_location)
    
    current_table.write.parquet("/FileStore/tables/northwind/trusted/"+file+".parquet")
      
    spark.read.parquet("/FileStore/tables/northwind/trusted/"+file+".parquet").createOrReplaceTempView(file+"_view")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS Date_Dim (
  date_id INT NOT NULL,
  Date DATE,
  Quarter INT,
  Day INT,
  Month INT,
  Year INT
);

INSERT INTO Date_Dim (date_id, Date, Quarter, Day, Month, Year)
SELECT 
  ROW_NUMBER() OVER (ORDER BY OrderDate),
  OrderDate,
  extract(QUARTER FROM OrderDate),
  extract(DAY FROM OrderDate),
  extract(MONTH FROM OrderDate),
  extract(YEAR FROM OrderDate)
FROM Orders_view;

num_affected_rows,num_inserted_rows
830,830


In [0]:
%sql
CREATE TABLE IF NOT EXISTS employee_dim 
(
  EmployeeID INT NOT NULL,
  LastName VARCHAR(20) NOT NULL,
  FirstName VARCHAR(10) NOT NULL,
  Region VARCHAR(15),
  City VARCHAR(15),
  Country VARCHAR(15)
);

INSERT INTO employee_dim (EmployeeID, LastName, FirstName, Region, City, Country)
SELECT 
  EmployeeID,
  LastName,
  FirstName,
  Region,
  City,
  Country
FROM Employees_view;



num_affected_rows,num_inserted_rows
9,9


In [0]:
%sql
CREATE TABLE IF NOT EXISTS product_dim 
(
  ProductID INT NOT NULL,
  ProductName VARCHAR(40) NOT NULL,
  CategoryID INT,
  QuantityPerUnit VARCHAR(20)
);

INSERT INTO product_dim (ProductID, ProductName, CategoryID, QuantityPerUnit)
SELECT DISTINCT
  ProductID, 
  ProductName, 
  CategoryID, 
  QuantityPerUnit
FROM Products_view;



num_affected_rows,num_inserted_rows
77,77


In [0]:
%sql
CREATE TABLE IF NOT EXISTS customer_dim 
(
  CustomerID CHAR(5) NOT NULL,
  ContactName VARCHAR(30), 
  CompanyName VARCHAR(40) NOT NULL,
  Region VARCHAR(15),
  City VARCHAR(15),
  Country VARCHAR(15)
);

INSERT INTO customer_dim (CustomerID, ContactName, CompanyName, Region, City, Country)
SELECT DISTINCT
  CustomerID, 
  ContactName, 
  CompanyName, 
  Region, 
  City, 
  Country
FROM Customers_view;



num_affected_rows,num_inserted_rows
91,91


In [0]:
%sql
CREATE TABLE IF NOT EXISTS order_fact 
(
  OrderID INT NOT NULL,
  OrderDate DATE,
  ProductID INT NOT NULL, 
  CustomerID CHAR(5),
  EmployeeID INT, 
  RequiredDate DATE, 
  ShippedDate DATE,
  ShipVia INT,
  Date DATE,
  Quantity SMALLINT NOT NULL,
  Discount FLOAT NOT NULL,
  UnitPrice FLOAT NOT NULL,
  Freight FLOAT,
  UnitsInStock SMALLINT,
  UnitsInOrder SMALLINT,
  ReorderLevel SMALLINT
);

INSERT INTO order_fact (OrderID, OrderDate, ProductID, CustomerID, EmployeeID, RequiredDate, ShippedDate, ShipVia, Date, Quantity, Discount, UnitPrice, Freight, UnitsInStock, UnitsInOrder, ReorderLevel)
SELECT DISTINCT
  o.OrderID, 
  o.OrderDate, 
  p.ProductID, 
  o.CustomerID,
  o.EmployeeID,
  o.RequiredDate, 
  o.ShippedDate,
  o.ShipVia,
  d.Date,
  od.Quantity,
  od.Discount,
  od.UnitPrice,
  o.Freight,
  p.UnitsInStock,
  p.UnitsOnOrder,
  p.ReorderLevel
FROM Orders_view o JOIN Order_Details__view od ON od.OrderID = o.OrderID JOIN Products_view p ON od.ProductID = p.ProductID JOIN date_dim d ON d.Date = o.OrderDate;


num_affected_rows,num_inserted_rows
2155,2155


# Questões

### Quais os 3 produtos MENOS vendidos?

In [0]:
%sql
SELECT SUM(odv.Quantity) as Quantity, odv.ProductID, pv.ProductName FROM Order_details__view as odv
INNER JOIN Products_view as pv
  on odv.ProductID = pv.ProductID
GROUP BY odv.ProductID, pv.ProductName
ORDER BY Quantity
LIMIT 3

Quantity,ProductID,ProductName
95.0,9,Mishi Kobe Niku
122.0,15,Genen Shouyu
125.0,37,Gravad lax


In [0]:
%sql
SELECT SUM(orf.Quantity) as Quantity, ProductName FROM Order_fact as orf
INNER JOIN Product_dim as pd
  ON orf.ProductID = pd.ProductID
GROUP BY ProductName
ORDER BY Quantity

Quantity,ProductName
95,Mishi Kobe Niku
122,Genen Shouyu
125,Gravad lax
138,Chocolade
184,Laughing Lumberjack Lager
235,Valkoinen suklaa
239,Louisiana Hot Spiced Okra
293,R�d Kaviar
297,Longlife Tofu
297,Mascarpone Fabioli


### Quais são os cinco clientes que MAIS compras fizeram? (quantidade)

In [0]:
%sql
SELECT COUNT(odv.Quantity) as Quantity, cv.CompanyName FROM Order_details__view as odv
INNER JOIN Orders_view as ov
  on odv.OrderID = ov.OrderID
INNER JOIN Customers_view as cv
  on ov.CustomerID = cv.CustomerID
GROUP BY cv.CompanyName
ORDER BY Quantity DESC
LIMIT 5

Quantity,CompanyName
116,Save-a-lot Markets
102,Ernst Handel
86,QUICK-Stop
71,Rattlesnake Canyon Grocery
55,Hungry Owl All-Night Grocers


In [0]:
%sql
SELECT COUNT(orf.OrderID) as Quantity, CompanyName FROM Order_fact as orf
INNER JOIN Customer_dim as cd
  ON orf.CustomerID = cd.CustomerID
GROUP BY CompanyName
ORDER BY Quantity DESC
LIMIT 5

Quantity,CompanyName
116,Save-a-lot Markets
102,Ernst Handel
86,QUICK-Stop
71,Rattlesnake Canyon Grocery
55,Hungry Owl All-Night Grocers


### Quais são os cinco clientes com MAIOR total gasto? (montante)

In [0]:
%sql
SELECT SUM(Quantity * UnitPrice) as Montante, CompanyName FROM Order_fact as orf
INNER JOIN Customer_dim as cd
  on orf.CustomerID = cd.CustomerID
GROUP BY CompanyName
ORDER BY Montante DESC
LIMIT 5

Montante,CompanyName
117483.38994598389,QUICK-Stop
115673.38977050781,Save-a-lot Markets
113236.67984008789,Ernst Handel
57317.390268325806,Hungry Owl All-Night Grocers
52245.90013694763,Rattlesnake Canyon Grocery


### Qual o melhor funcionário do último mês registrado? (total de vendas)

In [0]:
%sql
SELECT FirstName, SUM(Quantity * UnitPrice) as Prices FROM Order_fact as orf
INNER JOIN Employee_dim as em
  ON orf.EmployeeID = em.EmployeeID
WHERE MONTH(Date) = 5 and YEAR(Date) = 1998
GROUP BY FirstName
ORDER BY Prices DESC

FirstName,Prices
Nancy,7053.749994277954
Margaret,6275.0
Laura,3223.3599853515625
Andrew,2173.5
Robert,1173.0500144958496


### Quais as regiões (*País) com menos clientes cadastrados?

In [0]:
%sql
SELECT COUNT(CustomerID) as Clients, Country FROM Customer_dim
GROUP BY Country
ORDER BY Clients ASC
LIMIT 10